<a href="https://www.kaggle.com/code/utkarshx27/covid-tweet-classification-nb-vs-lstm?scriptVersionId=133277559" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv
/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score


import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [3]:
train = pd.read_csv("/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_train.csv",encoding="latin-1")
test = pd.read_csv("/kaggle/input/covid-19-nlp-text-classification/Corona_NLP_test.csv",encoding="latin-1")

In [4]:
train.head()

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,3799,48751,London,16-03-2020,@MeNyrbie @Phil_Gahan @Chrisitv https://t.co/i...,Neutral
1,3800,48752,UK,16-03-2020,advice Talk to your neighbours family to excha...,Positive
2,3801,48753,Vagabonds,16-03-2020,Coronavirus Australia: Woolworths to give elde...,Positive
3,3802,48754,NaN,16-03-2020,My food stock is not the only one which is emp...,Positive
4,3803,48755,NaN,16-03-2020,"Me, ready to go at supermarket during the #COV...",Extremely Negative


In [5]:
test.columns

Index(['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet',
       'Sentiment'],
      dtype='object')

In [6]:
train = train.loc[:, ['OriginalTweet', 'Sentiment']]
train = train.rename(columns={'OriginalTweet': 'tweet', 'Sentiment': 'emotion'})
test = test.loc[:, ['OriginalTweet', 'Sentiment']]
test = test.rename(columns={'OriginalTweet': 'tweet', 'Sentiment': 'emotion'})

In [7]:
train.columns

Index(['tweet', 'emotion'], dtype='object')

In [8]:
def cleaning(data):
    
    df = data.copy()
    
    df.columns = map(str.lower, df.columns)
    df = df[['tweet', 'emotion']]

    def remove_web_urls(text): return re.sub(r'https?://\S+', ' ', text)
    df['tweet'] = df['tweet'].apply(remove_web_urls)
    
    def remove_tags(text): return re.sub(r'@\w*', ' ' , text)
    df['tweet'] = df['tweet'].apply(remove_tags)
    
    def remove_hashtags(text): return re.sub(r'#\w*', ' ' , text)
    df['tweet'] = df['tweet'].apply(remove_hashtags)
    
    def remove_apostrophe(text): return re.sub(r"'s\b", "", text)
    df['tweet'] = df['tweet'].apply(remove_apostrophe)

    def remove_special_chars(text): return re.sub(r"[^a-zA-Z0-9\s]", ' ', text)
    df['tweet'] = df['tweet'].apply(remove_special_chars)
    
    def remove_number(text): return re.sub(r'[\d]', ' ', text)
    df['tweet'] = df['tweet'].apply(remove_number)
    
    df['tweet'] = df['tweet'].str.lower()
    return df
cleaned_train = cleaning(train)
cleaned_test = cleaning(test)

In [9]:
cleaned_train.shape, cleaned_test.shape

((41157, 2), (3798, 2))

In [10]:
train['emotion'].unique()

array(['Neutral', 'Positive', 'Extremely Negative', 'Negative',
       'Extremely Positive'], dtype=object)

In [11]:
cleaned_train["emotion"].value_counts()

Positive              11422
Negative               9917
Neutral                7713
Extremely Positive     6624
Extremely Negative     5481
Name: emotion, dtype: int64

In [12]:
cleaned_train['emotion'] = cleaned_train['emotion'].str.replace('Extremely Positive', 'Positive')
cleaned_train['emotion'] = cleaned_train['emotion'].str.replace('Extremely Negative', 'Negative')

cleaned_test['emotion'] = cleaned_test['emotion'].str.replace('Extremely Positive', 'Positive')
cleaned_test['emotion'] = cleaned_test['emotion'].str.replace('Extremely Negative', 'Negative')

In [13]:
cleaned_train["emotion"].value_counts(), cleaned_test["emotion"].value_counts()

(Positive    18046
 Negative    15398
 Neutral      7713
 Name: emotion, dtype: int64,
 Negative    1633
 Positive    1546
 Neutral      619
 Name: emotion, dtype: int64)

In [14]:
X = cleaned_train['tweet']
y = cleaned_train['emotion']
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [15]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((32925,), (8232,), (32925,), (8232,))

In [16]:
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_val_vectorized = vectorizer.transform(X_val)

In [17]:
model = MultinomialNB()
model.fit(X_train_vectorized, y_train)
y_pred = model.predict(X_val_vectorized)

accuracy = accuracy_score(y_val, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6922983479105929


In [18]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train)

X_train_sequences = tokenizer.texts_to_sequences(X_train)
X_val_sequences = tokenizer.texts_to_sequences(X_val)


max_sequence_length = 100 
X_train_padded = pad_sequences(X_train_sequences, maxlen=max_sequence_length)
X_val_padded = pad_sequences(X_val_sequences, maxlen=max_sequence_length)


label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)


model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=100, input_length=max_sequence_length))
model.add(LSTM(128))
model.add(Dense(5, activation='softmax'))  

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    mirrored_model = model


mirrored_model.fit(X_train_padded, y_train_encoded, validation_data=(X_val_padded, y_val_encoded), epochs=5, batch_size=32)


y_pred_probs = mirrored_model.predict(X_val_padded)
y_pred_encoded = tf.argmax(y_pred_probs, axis=1).numpy()


y_pred_labels = label_encoder.inverse_transform(y_pred_encoded)


accuracy = accuracy_score(y_val, y_pred_labels)
print("Accuracy:", accuracy)

Epoch 1/5
1029/1029 [==============================] - 50s 43ms/step - loss: 0.6693 - accuracy: 0.7048 - val_loss: 0.3984 - val_accuracy: 0.8678
Epoch 2/5
1029/1029 [==============================] - 15s 15ms/step - loss: 0.2804 - accuracy: 0.9094 - val_loss: 0.3523 - val_accuracy: 0.8827
Epoch 3/5
1029/1029 [==============================] - 12s 11ms/step - loss: 0.1765 - accuracy: 0.9448 - val_loss: 0.3855 - val_accuracy: 0.8768
Epoch 4/5
1029/1029 [==============================] - 11s 10ms/step - loss: 0.1164 - accuracy: 0.9626 - val_loss: 0.4224 - val_accuracy: 0.8744
Epoch 5/5
258/258 [==============================] - 1s 4ms/step
Accuracy: 0.874271137026239
